In [8]:
import os
import sys
import simulator as sim
import numpy as np
import datetime as dt
import time
from sklearn import svm

SEC = 1000000
MIN = 60 * SEC
DOLLAR = 1000000
CENT = 10000

DATA_PATH = "data/"

###############################################################################
# Default algorithm with best bid best ask
###############################################################################


class SVMAlgo:
    
    def __init__(self, session, date, ticker, start_time, end_time, generator, bld, nl):
        # Save session information
        self.session = session
        self.date = date
        self.tickers = ticker
        self.start_time = start_time
        self.end_time = end_time
        self.clf = generator
        self.bld_up = bld
        self.num_levels = nl
        self.order_size = 1
        # PnL and book value functions need to be tested when order_size is not 1
        self.mu = 1 * CENT
        self.ORDER_WAIT = 30
        print "date is {0}".format(date)
        # outstanding_orders is a dictionary, tickers are the keys and values are 
        # {"A": [ask_order_id, time_accepted], "B": [bid_order_id, time_accepted]}
        # if there are outstanding orders
        self.outstanding_orders = {}
        
        # signal is a dictionary : {symbol : symbol_signal}
        self.signal = {}
        
        # inventory is another dictionary: {symbol : inventory_size for symbol}
        # cash is another dictionary: {symbol : cash_balance for symbol}
        # bookvalue is another dictionary: {symbol : book value for symbol}
        self.inventory = {}
        self.cash = {}
        self.bookvalue = {}
        # self.inv_hist is a dict of numpy arrays, the arrays are as follows:
        # field for each column: time, symbol_cash, symbol_inventory, symbol_book_value, symbol_current_mid_price
        self.inv_hist = {}
        
        # Suscribe to tickers and set first timer
        # no trading on the 1st 15 minute: use to compute signal
        self.offset = 15
        num = (end_time - start_time) / MIN - self.offset
        self.obs_counter = 0
        
        self.hist_ordbook = np.zeros(((end_time - start_time) / MIN, num_levels * 2))
        for i in range((end_time - start_time) / MIN):
            self.session.add_timer(self.start_time + i * MIN, self.timer_get_orderbook_callback)
            
        for i in range(num):
            self.session.add_timer(self.start_time + (i + self.offset) * MIN, self.timer_OBS_callback)
        
        for sym_it in self.tickers:
            self.session.subscribe_ticker_all_feeds(sym_it)
            self.outstanding_orders[sym_it] = {}
            self.signal[sym_it] = self.get_signal(sym_it)
            self.inventory[sym_it] = 0
            self.cash[sym_it] = 0
            self.bookvalue[sym_it] = 0
            self.session.subscribe_event(sym_it, sim.ORDER_ACCEPTED, self.order_ack_callback)
            self.session.subscribe_event(sym_it, sim.ORDER_EXECUTED, self.order_exe_callback)
            self.session.subscribe_event(sym_it, sim.ORDER_CANCELED, self.cancelled_callback)
            self.session.subscribe_event(sym_it, sim.ORDER_REJECTED, self.rejected_callback)
            self.inv_hist[sym_it] = np.zeros((num, 5))
    
    def timer_get_orderbook_callback(self, time):
        book = self.session.get_book_levels(self.ticker, nlevels = self.num_levels)
        
        
    def timer_OBS_callback(self, time):
        #print "OBS callback: {0}, {1}".format(sim.micro_to_time(time), sim.micro_to_time(self.end_time))
        if (time >= self.end_time):
            self.end()
            return
        for sym_it in self.signal:
            curr_sig = self.get_signal(sym_it)
            print "Current signal is {0}, signal before is {1}".format(curr_sig, self.signal[sym_it])
            if (curr_sig != self.signal[sym_it]):
                self.cancel_orders(sym_it)
                self.stop_loss(sym_it)
                self.signal[sym_it] = curr_sig
            self.post_orders(sym_it)
        for sym_it in self.tickers:
            bid, ask = self.session.get_inside_market(sym_it)
            b_price = bid['price']
            a_price = ask['price']
            self.inv_hist[sym_it][self.obs_counter] = np.array([time, self.cash[sym_it], self.inventory[sym_it], self.bookvalue[sym_it], (a_price + b_price) / 2])
        
        self.obs_counter += 1    
            
            
    def post_orders(self, ticker):
        now = self.session.current_time()
        orders = self.session.get_orders_by_ticker(ticker)
        if not orders:
            bid, ask = self.session.get_inside_market(ticker)
            b_price = bid['price'] # = bestBidPrice()
            a_price = ask['price']
            self.session.add_order(ticker, sim.BUY, self.order_size, b_price + self.signal[ticker] * self.mu, exchange=sim.EXCH_INET)
            self.session.add_order(ticker, sim.SELL, self.order_size, a_price + self.signal[ticker] * self.mu, exchange=sim.EXCH_INET) 
            #print "Posted orders"
               
    
    def cancel_orders(self, ticker):
        now = self.session.current_time()
        #print "cancel order call back: it's now {0}".format(sim.micro_to_time(now))
        ord_info = self.outstanding_orders[ticker]
        if len(ord_info) == 0:
            #print "no order to cancel"
            return
        else:
            for side in ord_info:
                sim.cancel_order(ord_info[side][0])
                #print "cancelled order {0}".format(ord_info[side][0])
        
        
    def stop_loss(self, ticker):
        print "stop loss for ticker {0}".format(ticker)
        inv = self.inventory[ticker]
        if (inv == 0):
            return
        elif (inv > 0):
            # not the right way to send market order
            self.session.add_order(ticker, sim.SELL, inv, 1 * DOLLAR, exchange=sim.EXCH_INET)
            print "selling inventory"
        else:
            # not the right way to send market order
            self.session.add_order(ticker, sim.BUY, -inv, 999 * DOLLAR, exchange=sim.EXCH_INET)
            print "buying inventory"
        
        
    def timer_order_manage_callback(self, time):
        now = self.session.current_time()
        #print "order manage back: it's now {0}".format(sim.micro_to_time(now))
        for sym_it in self.outstanding_orders:
            ord_info = self.outstanding_orders[sym_it]
            if len(ord_info) == 0:
                continue
            else:
                for side in ord_info:
                    cancel_time = ord_info[side][1]
                    if (now < cancel_time):
                        #this may add duplicate timer event for different symbols
                        self.session.add_timer(cancel_time, self.timer_order_manage_callback)
                    else:
                        sim.cancel_order(ord_info[side][0])
                        #print "Cancel order {0}".format(ord_info[side][0])
         
        
    def rejected_callback(self, ticker, event_params):
        print "rejected!!!!"
        print event_params['rejected_orders']
    
    
    def order_ack_callback(self, ticker, event_params):
        time = self.session.current_time()
        #print "Ack Call back: it's now {0}".format(sim.micro_to_time(time))
        for order in event_params['accepted_orders']:
            ord_id = order['order_id']
            sym = order['ticker']
            side = order['side']
            t = order['time_accepted']
            self.outstanding_orders[sym][side] = [ord_id, t]
        #print "Order accepted. order id is {0}, symbol is {1}, side is {2}, time is {3}".format(ord_id, sym, side, sim.micro_to_time(time))
        if time < self.end_time:
            self.session.add_timer(t + self.ORDER_WAIT * SEC, self.timer_order_manage_callback)    
        
        
    def order_exe_callback(self, ticker, event_params):
        time = self.session.current_time()
        #print "Execution call_back: it's now {0}".format(sim.micro_to_time(time))
        for exe in event_params['executed_orders']:
            order = exe['order']
            ord_id = order['order_id']
            sym = order['ticker']
            side = order['side']
            t = order['time_placed']
            self.outstanding_orders[sym].pop(side)
            if (side == "B"):
                if self.inventory[sym] < 0 :
                    if -self.inventory[sym] >= exe['quantity_executed']:
                        proportion = exe['quantity_executed'] / float(-self.inventory[sym])
                        self.bookvalue[sym] -= self.bookvalue[sym] * proportion
                    else:
                        self.bookvalue[sym] = (self.inventory[sym] + exe['quantity_executed']) * exe['price_executed']
                else:
                    self.bookvalue[sym] += exe['quantity_executed'] * exe['price_executed']
                self.inventory[sym] += exe['quantity_executed'] 
                self.cash[sym] -= exe['quantity_executed'] * exe['price_executed']
                if time < self.end_time:
                    if "S" in self.outstanding_orders[sym]:
                        self.outstanding_orders[sym]["S"][1] = time + self.ORDER_WAIT * SEC
                        #print "extending timer for sell order"
            else:
                if self.inventory[sym] > 0 :
                    if self.inventory[sym] >= exe['quantity_executed']:
                        proportion = exe['quantity_executed'] / float(self.inventory[sym])
                        self.bookvalue[sym] -= self.bookvalue[sym] * proportion
                    else:
                        self.bookvalue[sym] = (self.inventory[sym] - exe['quantity_executed']) * exe['price_executed']
                else:
                    self.bookvalue[sym] -= exe['quantity_executed'] * exe['price_executed']
                self.inventory[sym] -= exe['quantity_executed'] 
                self.cash[sym] += exe['quantity_executed'] * exe['price_executed']
                if time < self.end_time:
                    if "B" in self.outstanding_orders[sym]:
                        self.outstanding_orders[sym]["B"][1] = time + self.ORDER_WAIT * SEC
                        #print "extending timer for buy order"
            #print "Order Executed. order id is {0}, symbol is {1}, side is {2}, time is {3}, dollar amount is {4}".format(ord_id, sym, side, sim.micro_to_time(time), exe['quantity_executed'] * exe['price_executed'])
    
    def cancelled_callback(self, ticker, event_params):
        time = self.session.current_time()
        #print "Cancelled call back: it's now {0}".format(sim.micro_to_time(time))
        for order in event_params['canceled_orders']:
            ord_id = order['order_id']
            sym = order['ticker']
            side = order['side']
            t = order['time_placed']
            self.outstanding_orders[sym].pop(side)
            #print "Order Cancelled. order id is {0}, symbol is {1}, side is {2}, time is {3}".format(ord_id, sym, side, sim.micro_to_time(time))
        
        
    #returns -1, 0, or 1 corresponding to markets going up, even, or down
    def get_signal(self, ticker):
        
        self.clf.predict()
        #return self.obs_counter % 3 - 1
    
    
    def end_algo(self, time):
        for sym_it in self.cash:
            print "PnL for {0} is {1}".format(sym_it, self.cash[sym_it] / float(DOLLAR))
        
        for sym_it in self.tickers:
            bid, ask = self.session.get_inside_market(sym_it)
            b_price = bid['price']
            a_price = ask['price']
            self.inv_hist[sym_it][self.obs_counter] = np.array([time, self.cash[sym_it], self.inventory[sym_it], self.bookvalue[sym_it], (a_price + b_price) / 2])
        
        obp_filename = "{0}{1}_output".format(DATA_PATH, self.date)
        np.savez_compressed(obp_filename, **self.inv_hist)
    
    def end(self):
        time = self.session.current_time()
        if time < sim.string_to_micro("18:00"):
            print "DONDONDON: it's now {0}".format(sim.micro_to_time(time))
            print "Done"
            for sym_it in self.outstanding_orders:
                self.cancel_orders(sym_it)
            print "cash: {0}".format(self.cash)
            for sym_it in self.tickers:
                print "inventory: {0}".format(self.inventory[sym_it])    
                self.stop_loss(sym_it)

            self.session.add_timer(time + MIN, self.end_algo)
            
        return 

###############################################################################
# Main
###############################################################################
symbols = ["IVV"]
start_time = sim.string_to_micro("10:00")
end_time = sim.string_to_micro("10:10")

files = os.listdir(DATA_PATH)
dates = []
for f in files:
    if f.endswith("_bookdata.txt"):
        dates.append(f.split('_')[1])
dates.sort()

sim_dates = dates[15:16]

start = time.time()
for date_it in sim_dates:
    last_dates = get_last_days(dates, date_it, 15)

    train_data = np.zeros(0)
    train_sig = np.zeros(0)

    for date in last_dates[0:-1]:
        data_filename = "{0}{1}_{2}_obp.txt".format(DATA_PATH, ticker, date)
        sig_filename = "{0}{1}_{2}_sig.txt".format(DATA_PATH, ticker, date)
        if train_data.shape[0] == 0:
            train_data = np.loadtxt(data_filename)
            train_sig = np.loadtxt(sig_filename)
        else:
            temp1 = np.loadtxt(data_filename)
            temp2 = np.loadtxt(sig_filename)
            train_data = np.concatenate((train_data, temp1), axis=0)
            train_sig = np.concatenate((train_sig, temp2), axis=0) 
    clf = svm.SVC(C = 1, gamma = 'auto', decision_function_shape='ovo')
    clf.fit(train_data, train_sig)
    buildup = [1, 2, 3, 5, 10, 15]
    num_levels = train_data.shape[1] / len(buildup)
    
    simul_trading = sim.Simulator(SVMAlgo)
    simul_trading.run(date_it, symbols, use_om=True, start_time=start_time, end_time=end_time, generator=clf, bld=buildup, nl=num_levels)
    
end = time.time()
print "Time elapsed: {0}".format(end - start)

date is 20150203
Current signal is -1, signal before is -1
Current signal is 0, signal before is -1
stop loss for ticker IVV
Current signal is 1, signal before is 0
stop loss for ticker IVV
Current signal is -1, signal before is 1
stop loss for ticker IVV
Current signal is 0, signal before is -1
stop loss for ticker IVV
Current signal is 1, signal before is 0
stop loss for ticker IVV
buying inventory
Current signal is -1, signal before is 1
stop loss for ticker IVV
Current signal is 0, signal before is -1
stop loss for ticker IVV
Current signal is 1, signal before is 0
stop loss for ticker IVV
DONDONDON: it's now 10:10:00.000197
Done
cash: {'IVV': -204820000}
inventory: 1
stop loss for ticker IVV
selling inventory
PnL for IVV is 0.0
Time elapsed: 33.5921900272
